In [33]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import datetime

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import accuracy_score


import tensorflow as tf


from tensorflow import keras
import seaborn as sbn

In [34]:
# import data TODO: Change the file path to the dataset this question needed
df = pd.read_csv('..\\source_data\\cf_gas_et_df.csv')
df

,week,oil_futures,cpi,us_pop,public_roads,us_urban_pop,oil_production,auto_sales,us_real_gdp,gasoline
0,2001-01-07,30.05,78.645985,283960.0,3951330.509,223120517.5,5944,17.629145,12999.5699,1.425
1,2001-01-14,32.19,68.158536,284011.5,3951560.019,223171897.9,5949,17.850691,12293.3405,1.474
2,2001-01-21,29.77,76.149426,284063.0,3951789.528,223223278.4,5912,16.499039,12708.8104,1.471
3,2001-01-28,31.19,76.116224,284114.5,3952019.038,223274658.9,5935,14.163210,12287.2265,1.460
4,2001-02-04,31.03,66.693151,284166.0,3952248.547,223326039.4,5929,16.917572,13353.2628,1.443
...,...,...,...,...,...,...,...,...,...,...
986,2019-12-01,59.20,112.084710,329314.0,4190039.827,270476205.2,12900,17.941803,19477.3091,2.575
987,2019-12-08,60.07,109.471205,329335.8,4189928.462,270513569.8,12800,14.872529,19750.1111,2.561
988,2019-12-15,60.44,108.183411,329357.6,4189817.096,270550934.3,12800,18.616565,20106.3904,2.536
989,2019-12-22,61.72,108.707988,329379.4,4189705.731,270588298.9,12900,16.962709,20396.2701,2.532


# Creating independent and dependent variables 

In [35]:
# calculate independent variables
periods = (1,2,3,5,10)
for i in periods:
    df['of_chg_{}'.format(i)] = df['oil_futures'].pct_change(periods = i)
    
# calculate dependent variable
df['gas_chg'] = df['gasoline'].pct_change(periods=1)

# eliminate the empty rows
df = df[11:]
df

,week,oil_futures,cpi,us_pop,public_roads,us_urban_pop,oil_production,auto_sales,us_real_gdp,gasoline,of_chg_1,of_chg_2,of_chg_3,of_chg_5,of_chg_10,gas_chg
11,2001-03-25,26.29,71.870113,284546.5,3953855.113,223685702.7,5868,17.606315,11982.2289,1.404,-0.036996,-0.016829,-0.061407,-0.094697,-0.183287,0.000000
12,2001-04-01,27.06,74.862456,284602.0,3954084.623,223737083.1,5883,16.641042,13936.6979,1.442,0.029289,-0.008791,0.011967,-0.028017,-0.091031,0.027066
13,2001-04-08,28.25,67.840486,284648.4,3954314.132,223788463.6,5867,16.423081,12809.3245,1.500,0.043976,0.074553,0.034799,0.008568,-0.094261,0.040222
14,2001-04-15,27.28,82.229017,284694.8,3954543.642,223839844.1,5855,16.794574,13474.2006,1.571,-0.034336,0.008130,0.037657,0.020194,-0.120851,0.047333
15,2001-04-22,28.27,76.824931,284741.2,3954773.151,223891224.5,5861,17.016378,13236.2220,1.619,0.036290,0.000708,0.044715,0.035531,-0.030521,0.030554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,2019-12-01,59.20,112.084710,329314.0,4190039.827,270476205.2,12900,17.941803,19477.3091,2.575,0.073047,0.024753,0.025641,0.053381,0.058845,-0.001551
987,2019-12-08,60.07,109.471205,329335.8,4189928.462,270513569.8,12800,14.872529,19750.1111,2.561,0.014696,0.088816,0.039813,0.049441,0.137474,-0.005437
988,2019-12-15,60.44,108.183411,329357.6,4189817.096,270550934.3,12800,18.616565,20106.3904,2.536,0.006159,0.020946,0.095523,0.047124,0.104936,-0.009762
989,2019-12-22,61.72,108.707988,329379.4,4189705.731,270588298.9,12900,16.962709,20396.2701,2.532,0.021178,0.027468,0.042568,0.068375,0.147639,-0.001577


# Seperating training, validation, and test datasets

In [36]:
# create list of independent variable's name TODO: add the appropriate variable names to the list
ind_var_name = ['of_chg_{}'.format(i) for i in periods]+['cpi','us_pop','public_roads',
                                                         'us_urban_pop','oil_production','auto_sales',
                                                         'us_real_gdp']
num_var =len(ind_var_name) # put number of variables here

# extract the values to X
X = df[['oil_futures'] + ind_var_name].values
X

array([[ 2.62900000e+01, -3.69963370e-02, -1.68287210e-02, ...,
         5.86800000e+03,  1.76063154e+01,  1.19822289e+04],
       [ 2.70600000e+01,  2.92887029e-02, -8.79120879e-03, ...,
         5.88300000e+03,  1.66410415e+01,  1.39366979e+04],
       [ 2.82500000e+01,  4.39763489e-02,  7.45530620e-02, ...,
         5.86700000e+03,  1.64230808e+01,  1.28093245e+04],
       ...,
       [ 6.04400000e+01,  6.15948061e-03,  2.09459459e-02, ...,
         1.28000000e+04,  1.86165647e+01,  2.01063904e+04],
       [ 6.17200000e+01,  2.11780278e-02,  2.74679541e-02, ...,
         1.29000000e+04,  1.69627090e+01,  2.03962701e+04],
       [ 6.30500000e+01,  2.15489307e-02,  4.31833223e-02, ...,
         1.29000000e+04,  1.78387504e+01,  1.88128854e+04]])

In [37]:
# create y for regressor, y_c for classifier
y = df['gas_chg']
y_c = (df['gas_chg'] > 0).values.astype('int')

# set training, validation, and test criterias
training_ratio = 0.7
validation_ratio = 0.15
test_ratio = 0.15

# check correctness of X and y
X.shape, y.shape, y_c.shape

((980, 13), (980,), (980,))

In [38]:
# create training, validation, and test sets for regressor
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_ratio, random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = validation_ratio/(validation_ratio + training_ratio), random_state = 42)
print("X_train shape:\t", X_train.shape)
print("X_test shape:\t", X_test.shape)
print("X_val shape:\t", X_val.shape)
print("y_train shape:\t", y_train.shape)
print("y_val shape:\t", y_val.shape)
print("y_test shape:\t", y_test.shape)

X_train shape:	 (686, 13)
X_test shape:	 (147, 13)
X_val shape:	 (147, 13)
y_train shape:	 (686,)
y_val shape:	 (147,)
y_test shape:	 (147,)


In [39]:
# create training, validation, and test sets for classifier
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X, y_c, test_size = test_ratio, random_state = 42)
X_train_c, X_val_c, y_train_c, y_val_c = train_test_split(X_train_c, y_train_c, test_size = validation_ratio/(validation_ratio + training_ratio), random_state = 42)
print("X_train_c shape:", X_train_c.shape)
print("X_test_c shape:\t", X_test_c.shape)
print("X_val_c shape:\t", X_val_c.shape)
print("y_train_c shape:", y_train_c.shape)
print("y_val_c shape:\t", y_val_c.shape)
print("y_test_c shape:\t", y_test_c.shape)

X_train_c shape: (686, 13)
X_test_c shape:	 (147, 13)
X_val_c shape:	 (147, 13)
y_train_c shape: (686,)
y_val_c shape:	 (147,)
y_test_c shape:	 (147,)


# Principal Component Analysis (PCA)

In [40]:
# normalization for regressor set
for i in range(X_train.shape[1]):
    X_train_mean = X_train[:, i]. mean()
    X_train_std = X_train[:, i].std()
    X_train[:, i] = (X_train[:,i] - X_train_mean) / X_train_std
    X_test[:,i] = (X_test[:,i] - X_train_mean) / X_train_std
    X_val[:, i] = (X_val[:,i] - X_train_mean) / X_train_std

print("X_train shape:\t", X_train.shape)
print("X_val shape:\t", X_val.shape)

X_train shape:	 (686, 13)
X_val shape:	 (147, 13)


In [41]:
# normalization for classifier set
for i in range(X_train_c.shape[1]):
    X_train_c_mean = X_train_c[:, i]. mean()
    X_train_c_std = X_train_c[:, i].std()
    X_train_c[:, i] = (X_train_c[:,i] - X_train_c_mean) / X_train_c_std
    X_test_c[:,i] = (X_test_c[:,i] - X_train_c_mean) / X_train_c_std
    X_val_c[:, i] = (X_val_c[:,i] - X_train_c_mean) / X_train_c_std

print("X_train_c shape:", X_train_c.shape)
print("X_val_c shape:\t", X_val_c.shape)

X_train_c shape: (686, 13)
X_val_c shape:	 (147, 13)


In [42]:
# PCA for regressor
# fit on training set
scaler = StandardScaler()
scaler.fit(X_train)

# apply transform to training and test set
X_train_pca = scaler.transform(X_train)
X_test_pca = scaler.transform(X_test)
X_val_pca = scaler.transform(X_val)

# set PCA value
pca = PCA(0.95)

# fit PCA training set
pca.fit(X_train_pca)

# apply transform to training and test set
X_train_pca = pca.transform(X_train_pca)
X_test_pca = pca.transform(X_test_pca)
X_val_pca = pca.transform(X_val_pca)
print("X_train_pca shape:\t", X_train_pca.shape)
print("X_test_pca shape:\t", X_test_pca.shape)
print("X_val_pca shape:\t", X_val_pca.shape)
print("y_train shape:\t", y_train.shape)
print("y_val shape:\t", y_val.shape)
print("y_test shape:\t", y_test.shape)

X_train_pca shape:	 (686, 7)
X_test_pca shape:	 (147, 7)
X_val_pca shape:	 (147, 7)
y_train shape:	 (686,)
y_val shape:	 (147,)
y_test shape:	 (147,)


In [43]:
# PCA for classifier
# fit on training set_c
scaler = StandardScaler()
scaler.fit(X_train_c)

# apply transform to training and test set
X_train_c_pca = scaler.transform(X_train_c)
X_test_c_pca = scaler.transform(X_test_c)
X_val_c_pca = scaler.transform(X_val_c)

# set PCA value
pca = PCA(0.95)

# fit PCA training set
pca.fit(X_train_c_pca)

# apply transform to training and test set
X_train_c_pca = pca.transform(X_train_c_pca)
X_test_c_pca = pca.transform(X_test_c_pca)
X_val_c_pca = pca.transform(X_val_c_pca)
print("X_train_c_pca shape:\t", X_train_c_pca.shape)
print("X_test_c_pca shape:\t", X_test_c_pca.shape)
print("X_val_c_pca shape:\t", X_val_c_pca.shape)
print("y_train_c shape:", y_train_c.shape)
print("y_val_c shape:\t", y_val_c.shape)
print("y_test_c shape:\t", y_test_c.shape)

X_train_c_pca shape:	 (686, 7)
X_test_c_pca shape:	 (147, 7)
X_val_c_pca shape:	 (147, 7)
y_train_c shape: (686,)
y_val_c shape:	 (147,)
y_test_c shape:	 (147,)


In [44]:
num_var = X_train_c_pca.shape[1]
num_var

7

# Multiple linear regression

In [45]:
multiple_reg = LinearRegression().fit(X_train_pca, y_train)
score_multiple_reg = multiple_reg.score(X_test_pca, y_test)
print('score_multiple_reg:', "%.4f"%score_multiple_reg)

score_multiple_reg: 0.3156


# Multiple polynomial regression

In [46]:
#to prove this is the correct polynomial regression, set degree to 1
poly = PolynomialFeatures(1)
# polynomial transform
X_train_poly = poly.fit_transform(X_train_pca, y_train)
X_val_poly = poly.fit_transform(X_val_pca,y_val)
X_test_poly = poly.fit_transform(X_test_pca,y_test)

# fit to linear regression
polyreg = LinearRegression().fit(X_train_poly, y_train)

# predict using liear regression
score_poly_reg = polyreg.score(X_test_poly, y_test)
score_poly_reg

0.31558600921245605

In [47]:
# use val to find the appropriate degree
mpr = dict()
for i in range(2,6):
    degree=i
    poly = PolynomialFeatures(degree)
    # polynomial transform
    X_train_poly = poly.fit_transform(X_train_pca, y_train)
    X_val_poly = poly.fit_transform(X_val_pca,y_val)
    X_test_poly = poly.fit_transform(X_test_pca,y_test)

    # fit to linear regression
    polyreg = LinearRegression().fit(X_train_poly, y_train)

    # predict using liear regression
    score_poly_reg = polyreg.score(X_val_poly, y_val)
    print('score_poly_reg, degree',i,':', "%.4f"%score_poly_reg)
    mpr[f'{i}'] = score_poly_reg
    
max_degree = max(mpr, key = mpr.get)
print('max degree:', max_degree)

score_poly_reg, degree 2 : 0.4045
score_poly_reg, degree 3 : 0.3714
score_poly_reg, degree 4 : -23.8394
score_poly_reg, degree 5 : -10110.0897
max degree: 2


In [48]:
poly = PolynomialFeatures(int(max_degree))
# polynomial transform
X_train_poly = poly.fit_transform(X_train_pca, y_train)
X_val_poly = poly.fit_transform(X_val_pca,y_val)
X_test_poly = poly.fit_transform(X_test_pca,y_test)

# fit to linear regression
polyreg = LinearRegression().fit(X_train_poly, y_train)

# predict using liear regression
score_poly_reg = polyreg.score(X_test_poly, y_test)
score_poly_reg

0.1413812162680923

# Random forest classifier

In [49]:
# testing for different max depth
rfc = dict()
for i in range (1, 15):
    clf = RandomForestClassifier(max_depth=i, random_state=0).fit(X_train_c_pca, y_train_c)
    y_pred = clf.predict(X_val_c_pca)
    score_clf = accuracy_score(y_val_c, y_pred)
    rfc[f'{i}'] = score_clf
    #print('score_clf_',i,":", "%.4f"%score_clf)
max_depth = max(rfc, key=rfc.get)
print('max depth:', max_depth)

max depth: 13


In [50]:
# choose max_depth = 4
clf = RandomForestClassifier(max_depth=int(max_depth), random_state=0).fit(X_train_c_pca, y_train_c)
score_rf_clf = clf.score(X_test_c_pca, y_test_c)
print('score_rf_clf:', "%.4f"%score_rf_clf)

score_rf_clf: 0.6871


# Random forest regressor

In [51]:
clf_reg = RandomForestRegressor().fit(X_train_pca, y_train)
score_rf_reg = clf_reg.score(X_test_pca, y_test)
print('score_rf_reg:', "%.4f"%score_rf_reg)

score_rf_reg: 0.3369


# Logistic regression

In [52]:
logistic_model = LogisticRegression(penalty = 'l1', solver='liblinear', random_state=0).fit(X_train_c_pca, y_train_c)
score_logistic = logistic_model.score(X_test_c_pca, y_test_c)
print('score_logistic:', "%.4f"%score_logistic)

score_logistic: 0.7279


# Gradient boosting classifier

In [56]:
gra_clf = GradientBoostingClassifier(n_estimators = 200, learning_rate=0.50, max_depth=5, random_state=0).fit(X_train_c_pca, y_train_c)
score_gb_clf = gra_clf.score(X_test_c_pca, y_test_c)
score_gb_clf

0.7142857142857143

# Gradient boosting regressor 

In [58]:
gra_reg = GradientBoostingRegressor(random_state=0).fit(X_train_pca, y_train)
score_gb_reg = gra_reg.score(X_test_pca, y_test)
score_gb_reg

0.2750461231160386

# XGBoost classifier

In [23]:
xgb_clf = XGBClassifier().fit(X_train_c_pca, y_train_c)
score_xgb_clf = xgb_clf.score(X_test_c_pca, y_test_c)
score_xgb_clf

[16:04:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\lemon\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.7074829931972789

# XGBoost regressor

In [24]:
xgb_reg = XGBRegressor(random_state = 0).fit(X_train_pca, y_train)
score_xgb_reg = xgb_reg.score(X_test_pca, y_test)
score_xgb_reg

0.15949228182688358

# AdaBoost classifier

In [25]:
adb_clf = AdaBoostClassifier().fit(X_train_c_pca, y_train_c)
score_adb_clf = adb_clf.score(X_test_c_pca, y_test_c)
score_adb_clf

0.7210884353741497

# AdaBoost regressor

In [26]:
adb_reg = AdaBoostRegressor().fit(X_train_pca, y_train)
score_adb_reg = adb_reg.score(X_test_pca, y_test)
score_adb_reg

0.19282523856116418

# Support vector classification

In [27]:
# Try different kernels
kernels = ('linear', 'poly', 'rbf')

for k in kernels:
    svm_clf = SVC(kernel=k).fit(X_train_c_pca, y_train_c)
    y_pred = svm_clf.predict(X_test_c_pca)
    vars()[f'score_svc_{k}']= accuracy_score(y_test_c, y_pred)
    
print(score_svc_linear)
print(score_svc_poly)
print(score_svc_rbf)

0.7278911564625851
0.6938775510204082
0.6870748299319728


# Gaussian Naive Bayes

In [28]:
gnb_clf = GaussianNB().fit(X_train_c_pca, y_train_c)
score_gnb = gnb_clf.score(X_test_c_pca, y_test_c)
print(score_gnb)

0.7210884353741497


# Neuro networks

In [29]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(10, input_dim=num_var, activation='relu'))
model.add(keras.layers.Dense(5, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train_c_pca, y_train_c, epochs=50, validation_data=(X_val_c_pca, y_val_c))

Epoch 1/50
22/22 [==============================] - 1s 43ms/step - loss: 1.1070 - accuracy: 0.5033 - val_loss: 1.1399 - val_accuracy: 0.4286
Epoch 2/50
22/22 [==============================] - 0s 2ms/step - loss: 0.9489 - accuracy: 0.4839 - val_loss: 0.9467 - val_accuracy: 0.4422
Epoch 3/50
22/22 [==============================] - 0s 2ms/step - loss: 0.8010 - accuracy: 0.5583 - val_loss: 0.8297 - val_accuracy: 0.4558
Epoch 4/50
22/22 [==============================] - 0s 2ms/step - loss: 0.7349 - accuracy: 0.5469 - val_loss: 0.7572 - val_accuracy: 0.5306
Epoch 5/50
22/22 [==============================] - 0s 2ms/step - loss: 0.6962 - accuracy: 0.5660 - val_loss: 0.7255 - val_accuracy: 0.5510
Epoch 6/50
22/22 [==============================] - 0s 2ms/step - loss: 0.6809 - accuracy: 0.5722 - val_loss: 0.7026 - val_accuracy: 0.5578
Epoch 7/50
22/22 [==============================] - 0s 2ms/step - loss: 0.6541 - accuracy: 0.6048 - val_loss: 0.6845 - val_accuracy: 0.5850
Epoch 8/50
22/22 [=

In [30]:
loss, score_neuro = model.evaluate(X_test_c_pca, y_test_c)
print(score_neuro)

5/5 [==============================] - 0s 1ms/step - loss: 0.5880 - accuracy: 0.6735
0.6734693646430969


# Summary

In [31]:
reg = {
    'Multiple linear regression': score_multiple_reg,
    'Multiple polynomial regression': score_poly_reg,
    'Random forest regressor': score_rf_reg,
    'Gradient boosting regressor': score_gb_reg,
    'XGBoost regressor': score_xgb_reg,
    'AdaBoost regressor': score_adb_reg
}
reg_df = pd.DataFrame.from_dict(reg, orient='index', columns = ['Score'])
reg_df.index.name = 'Regression models'
#reg_df.sort_values(by=['Score'], ascending=False)
reg_df

,Score
Regression models,
Multiple linear regression,0.315586
Multiple polynomial regression,0.141381
Random forest regressor,0.371631
Gradient boosting regressor,0.275046
XGBoost regressor,0.159492
AdaBoost regressor,0.192825


In [32]:
clf = {
    'Logistic regression': score_logistic,
    'Random forest classifier': score_rf_clf,
    'Gradient boosting classifier': score_gb_clf,
    'XGBoost classifier': score_xgb_clf,
    'AdaBoost classifier': score_adb_clf,
    'Gaussian naive Bayes': score_gnb, 
    'SVC linear': score_svc_linear,
    'SVC polynomial': score_svc_poly, 
    'SVC RBF': score_svc_rbf,
    'Neuro network' : score_neuro
}
clf_df = pd.DataFrame.from_dict(clf, orient='index',columns = ['Score'])
clf_df.index.name = 'Classifier models'
#clf_df.sort_values(by=['Score'], ascending=False)
clf_df

,Score
Classifier models,
Logistic regression,0.727891
Random forest classifier,0.687075
Gradient boosting classifier,0.714286
XGBoost classifier,0.707483
AdaBoost classifier,0.721088
Gaussian naive Bayes,0.721088
SVC linear,0.727891
SVC polynomial,0.693878
SVC RBF,0.687075
